In [1]:
# Cài đặt thư viện
!pip install pandas "scikit-surprise==1.1.3" boto3 tqdm tabulate scikit-learn

import pandas as pd
import boto3
import pickle
import time
import numpy as np # <<< PHẢI CÓ DÒNG NÀY
from tqdm.notebook import tqdm
from surprise import Reader, Dataset, KNNBasic, SVD # <<< THÊM SVD
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity

# --- Cấu hình (THAY ĐỔI THEO CÀI ĐẶT CỦA BẠN) ---
S3_BUCKET_NAME = "movielens-25m-project" # <<< THAY TÊN BUCKET CỦA BẠN
AWS_REGION = "us-east-1"                           # <<< THAY REGION CỦA BẠN
RATINGS_FILE = "ratings.csv"
MOVIES_FILE = "movies.csv"
MOVIES_DF_KEY = "model/movies_df.pkl" # Đường dẫn lưu file tra cứu tên phim lên S3
DYNAMO_TABLE_NAME = "MovieRecommendations"

# Cấu hình S3 và DynamoDB
s3 = boto3.client('s3')
dynamodb = boto3.resource('dynamodb', region_name=AWS_REGION)
table = dynamodb.Table(DYNAMO_TABLE_NAME)

print("Cài đặt và import hoàn tất (đã thêm tqdm và cross_validate).")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/surprise/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution


Cài đặt và import hoàn tất (đã thêm tqdm và cross_validate).


In [2]:
print("Đang tải dữ liệu từ S3...")
s3.download_file(S3_BUCKET_NAME, RATINGS_FILE, RATINGS_FILE)
s3.download_file(S3_BUCKET_NAME, MOVIES_FILE, MOVIES_FILE)
print("Tải xong.")

print("Đang đọc CSV... (Bước này sẽ dùng nhiều RAM)")
# Đọc toàn bộ
ratings_df = pd.read_csv(RATINGS_FILE)
movies_df = pd.read_csv(MOVIES_FILE)

print(f"Đã tải {len(ratings_df)} lượt ratings.")
print(f"Đã tải {len(movies_df)} thông tin phim.")

Đang tải dữ liệu từ S3...
Tải xong.
Đang đọc CSV... (Bước này sẽ dùng nhiều RAM)
Đã tải 25000095 lượt ratings.
Đã tải 62423 thông tin phim.


In [3]:
# --- GIẢI QUYẾT MỤC 4.3: TRỰC QUAN KẾT QUẢ TRÊN CÁC LOẠI MODEL ---

print("Bắt đầu Mục 4.3: So sánh SVD vs KNNBasic (trên 1M sample)...")
reader = Reader(rating_scale=(0.5, 5.0))
sample_df = ratings_df.sample(n=1000000, random_state=1)
data_sample = Dataset.load_from_df(sample_df[['userId', 'movieId', 'rating']], reader)

print("Định nghĩa 2 mô hình để so sánh:")

# Model 1: SVD (Matrix Factorization)
algo_svd = SVD(n_factors=50, random_state=1) # n_factors=50 (nhẹ)

# Model 2: KNNBasic (Item-based)
sim_cosine = {'name': 'cosine', 'user_based': False}
algo_knn = KNNBasic(k=40, sim_options=sim_cosine)

# Chạy đánh giá chéo 3-fold (cv=3)
print("\nĐang chạy Cross-Validation cho Model 1 (SVD)...")
cv_svd = cross_validate(algo_svd, data_sample, measures=['RMSE', 'MAE'], cv=3, verbose=True)

print("\nĐang chạy Cross-Validation cho Model 2 (KNNBasic)...")
cv_knn = cross_validate(algo_knn, data_sample, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Trực quan kết quả
results = {
    'Loại Model': ['SVD (n_factors=50)', 'KNN (Cosine, k=40)'],
    'Chỉ số RMSE (Trung bình)': [cv_svd['test_rmse'].mean(), cv_knn['test_rmse'].mean()],
    'Chỉ số MAE (Trung bình)': [cv_svd['test_mae'].mean(), cv_knn['test_mae'].mean()],
    'Thời gian Fit (giây)': [sum(cv_svd['fit_time']) / len(cv_svd['fit_time']), sum(cv_knn['fit_time']) / len(cv_knn['fit_time'])]
}
results_df = pd.DataFrame(results)

print("\n--- KẾT QUẢ TRỰC QUAN (MỤC 4.3) ---")
print(results_df.to_markdown(index=False))

print(f"\n=> Phân tích: SVD thường có RMSE tốt hơn và fit nhanh hơn.")
print("   Cell tiếp theo sẽ huấn luyện SVD trên TOÀN BỘ 25M data.")

Bắt đầu Mục 4.3: So sánh SVD vs KNNBasic (trên 1M sample)...
Định nghĩa 2 mô hình để so sánh:

Đang chạy Cross-Validation cho Model 1 (SVD)...
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9150  0.9151  0.9118  0.9140  0.0015  
MAE (testset)     0.7033  0.7040  0.7018  0.7030  0.0009  
Fit time          8.21    8.70    8.56    8.49    0.20    
Test time         2.88    2.96    2.89    2.91    0.04    

Đang chạy Cross-Validation cho Model 2 (KNNBasic)...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.0857  1.0846  1.0856  1.0853  0.0005  
MAE (testset)     0.8251  0.8243  0.8238  0.8244 

In [4]:
# --- GIẢI QUYẾT MỤC 4.1: HUẤN LUYỆN MODEL SVD TRÊN TOÀN BỘ DATA ---

print("Bắt đầu chuẩn bị dữ liệu (Full 25M) cho SVD...")
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)
trainset = data.build_full_trainset()
print("Build full trainset 25M hoàn tất.")

print("Bắt đầu huấn luyện mô hình SVD (n_factors=100) trên TOÀN BỘ dữ liệu...")
algo = SVD(n_factors=100, n_epochs=25, random_state=1) 

start_time = time.time()
algo.fit(trainset)
end_time = time.time()
print(f"Huấn luyện SVD xong! Thời gian huấn luyện: {(end_time - start_time) / 60:.2f} phút.")

# Lấy ma trận Item Factors (movies x 100 factors)
# Chúng ta sẽ lưu biến này để Cell 5 dùng
item_factors = algo.qi 
print(f"Đã lấy ma trận Item Factors, kích thước: {item_factors.shape}")

# --- XÓA BỎ PHẦN GÂY CRASH ---
# (Đã xóa dòng sim_matrix = cosine_similarity(item_factors))
print("Bỏ qua việc tính toán ma trận tương đồng đầy đủ để tiết kiệm RAM.")
# -----------------------------
    
# Lưu movies_df (để tra cứu tên) lên S3 cho API sử dụng
print("Đang lưu movies_df.pkl lên S3...")
movies_df.to_pickle('movies_df.pkl')
s3.upload_file('movies_df.pkl', S3_BUCKET_NAME, MOVIES_DF_KEY)
print("Đã lưu movies_df.pkl lên S3.")

Bắt đầu chuẩn bị dữ liệu (Full 25M) cho SVD...
Build full trainset 25M hoàn tất.
Bắt đầu huấn luyện mô hình SVD (n_factors=100) trên TOÀN BỘ dữ liệu...
Huấn luyện SVD xong! Thời gian huấn luyện: 4.96 phút.
Đã lấy ma trận Item Factors, kích thước: (59047, 100)
Bỏ qua việc tính toán ma trận tương đồng đầy đủ để tiết kiệm RAM.
Đang lưu movies_df.pkl lên S3...
Đã lưu movies_df.pkl lên S3.


In [5]:
print("Bắt đầu tính toán trước (JIT) và lưu vào DynamoDB...")
all_movie_raw_ids = [trainset.to_raw_iid(inner_id) for inner_id in trainset.all_items()]
print(f"Tổng số phim cần tính toán: {len(all_movie_raw_ids)}")

item_count = 0
start_batch_time = time.time()

with table.batch_writer() as batch:
    print("Bắt đầu ghi vào DynamoDB (có progress bar):")
    # item_factors và trainset được lấy từ Cell 4
    
    for movie_raw_id in tqdm(all_movie_raw_ids, desc="Đang xử lý phim"):
        try:
            # 1. Chuyển ID gốc -> ID nội bộ
            movie_inner_id = trainset.to_inner_iid(movie_raw_id)
            
            # 2. Lấy vector đặc trưng (100,) của 1 phim
            movie_vector = item_factors[movie_inner_id]
            
            # 3. --- TÍNH TOÁN JIT (Just-In-Time) ---
            # Tính độ tương đồng của 1 phim này với TẤT CẢ các phim khác
            # Reshape (1, 100) để tính (1 vs 59047) -> output (1, 59047)
            movie_sims_vector = cosine_similarity(
                movie_vector.reshape(1, -1), # Vector của phim này
                item_factors                 # Ma trận của tất cả phim
            )[0] # Lấy hàng đầu tiên (và duy nhất)
            # ------------------------------------
            
            # 4. Sắp xếp vector này và lấy index của 11 phim (phim đầu tiên là chính nó)
            neighbors_inner_ids = np.argsort(movie_sims_vector)[-11:][::-1]
            
            # 5. Chuyển ID nội bộ -> ID gốc
            neighbors_raw_ids = [trainset.to_raw_iid(inner_id) for inner_id in neighbors_inner_ids]
            
            # 6. Lọc (bỏ phim đầu tiên, là chính nó) và chuyển sang string
            recommendations = [str(r) for r in neighbors_raw_ids[1:11]]
            
            item_data = {
                'movieId': str(movie_raw_id),
                'recommendations': recommendations 
            }
            batch.put_item(Item=item_data)
            item_count += 1
        except Exception as e:
            # Bỏ qua các phim không có trong trainset (lỗi "Not in trainset")
            pass 

end_batch_time = time.time()
print(f"\nHOÀN TẤT! Đã ghi {item_count} mục vào DynamoDB.")
print(f"Tổng thời gian ghi: {(end_batch_time - start_batch_time) / 60:.2f} phút.")

Bắt đầu tính toán trước (JIT) và lưu vào DynamoDB...
Tổng số phim cần tính toán: 59047
Bắt đầu ghi vào DynamoDB (có progress bar):


Đang xử lý phim:   0%|          | 0/59047 [00:00<?, ?it/s]


HOÀN TẤT! Đã ghi 59047 mục vào DynamoDB.
Tổng thời gian ghi: 39.98 phút.
